In [28]:
import json
import os
import pandas as pd
from scipy import stats

In [2]:
nodes = ["start", "count", "gccount1", "gccount2", "atcount1", "atcount2", "gcsum", "atsum", "gcratio"]

base_dir = "/home/ubuntu/tmp/y_fasta/"

# concatenator breaks the audit log: https://github.com/scipipe/scipipe/issues/127
# so we need to trace audit up to concatenator
# as well as each audit files leading up to concatenator
# annoying, but alas!
audit_files = ["gcratio.txt.audit.json",
               "chry.fa.gccnt1.audit.json", "chry.fa.gccnt2.audit.json",
               "chry.fa.atcnt1.audit.json", "chry.fa.atcnt2.audit.json"]

n_repeats = 20
n_records = 30


In [3]:
def load_dataset(mode, repeat_id):
    if mode not in ['normal', 'break_count', 'break_start']:
        raise ValueError(f'Wrong mode {mode} given')
    
    if repeat_id < 1 or repeat_id > n_repeats:
        raise ValueError(f'Wrong repeat id {repeat_id} given')

    experiment_dir = os.path.join(base_dir, 'experiment', mode, f'repeat_{repeat_id}')    
    all_records = []

    for i in range(1, n_records+1):
        raw_data = {}
        for final_audit_file in audit_files:
            with open(os.path.join(experiment_dir, str(i), final_audit_file)) as f:
                audit_data = json.load(f)
            
            audit_data_queue = [audit_data]
            while audit_data_queue:
                audit_data = audit_data_queue.pop(0)
                if audit_data["ProcessName"] in nodes:
                    # we assume all our nodes output exactly one file
                    # Big Assumption!
                    data_file_name = next(iter(audit_data["OutFiles"].values()))
                    with open(os.path.join(experiment_dir, str(i), data_file_name)) as data_file:
                        line = data_file.readline()

                    # calc may output tilde for division
                    # so we need to remove it, plus any whitespaces
                    value = float(line.strip().strip('~'))
                    raw_data[audit_data["ProcessName"]] = value
                
                for upstream_data in audit_data["Upstream"].values():
                    if upstream_data:
                        audit_data_queue.append(upstream_data)
        
        all_records.append(raw_data)

    return pd.DataFrame(all_records)


In [12]:
import networkx as nx, numpy as np, pandas as pd
from dowhy import gcm

In [13]:
causal_graph = nx.DiGraph([
    ('count', 'gccount1'),
    ('count', 'gccount2'),
    ('count', 'atcount1'),
    ('count', 'atcount2'),
    ('start', 'gccount1'),
    ('start', 'gccount2'),
    ('start', 'atcount1'),
    ('start', 'atcount2'),

    ('gccount1', 'gcsum'),
    ('atcount1', 'atsum'),
    ('gccount2', 'gcsum'),
    ('atcount2', 'atsum'),

    ('gcsum', 'gcratio'),
    ('atsum', 'gcratio'),
])

In [25]:
def compute_attributions(break_mode):
    attributions_per_experiment = []
    for i in range(1, n_repeats+1):
        normal_df = load_dataset('normal', i)
        break_df = load_dataset(break_mode, i)

        causal_model = gcm.ProbabilisticCausalModel(causal_graph)
        causal_model.set_causal_mechanism('count', gcm.EmpiricalDistribution())
        causal_model.set_causal_mechanism('start', gcm.EmpiricalDistribution())
        causal_model.set_causal_mechanism('gccount1', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
        causal_model.set_causal_mechanism('gccount2', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
        causal_model.set_causal_mechanism('atcount1', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
        causal_model.set_causal_mechanism('atcount2', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
        causal_model.set_causal_mechanism('gcsum', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
        causal_model.set_causal_mechanism('atsum', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
        causal_model.set_causal_mechanism('gcratio', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

        attributions = gcm.distribution_change(causal_model, normal_df, break_df, 'gcratio')
        attributions_per_experiment.append(attributions)
    
    return pd.DataFrame(attributions_per_experiment)

In [30]:
break_count_attr_df = compute_attributions('break_count')

Estimating Shapley Values. Average change of Shapley values in run 15 (75 evaluated permutations): -90.42880993625782%: 100%|██████████| 1/1 [00:06<00:00,  6.11s/it]
Estimating Shapley Values. Average change of Shapley values in run 14 (70 evaluated permutations): -2.7327459372430347%: 100%|██████████| 1/1 [00:01<00:00,  1.85s/it]
Estimating Shapley Values. Average change of Shapley values in run 10 (50 evaluated permutations): -4.152115919531976%: 100%|██████████| 1/1 [00:01<00:00,  1.44s/it]
Estimating Shapley Values. Average change of Shapley values in run 7 (35 evaluated permutations): -5.693890236030744%: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]
Estimating Shapley Values. Average change of Shapley values in run 16 (80 evaluated permutations): -18.99880166432163%: 100%|██████████| 1/1 [00:02<00:00,  2.33s/it]
Estimating Shapley Values. Average change of Shapley values in run 4 (20 evaluated permutations): -37.8239389990138%: 100%|██████████| 1/1 [00:00<00:00,  1.48it/s]
Estima

In [32]:
for column in nodes:
    if column == "count":
        continue
    
    test_result = stats.ttest_ind(break_count_attr_df["count"].to_list(),
                                  break_count_attr_df[column].to_list(),
                                  equal_var=False)
    
    print(f"{column}: p-value = {test_result.pvalue}")



start: p-value = 2.6093681727971215e-12
gccount1: p-value = 3.068260538322675e-12
gccount2: p-value = 6.5020863015428874e-12
atcount1: p-value = 2.390795330409867e-12
atcount2: p-value = 2.3327583848798365e-12
gcsum: p-value = 2.469054892496614e-12
atsum: p-value = 2.5011572318531435e-12
gcratio: p-value = 2.0713525132602816e-07


In [42]:
break_start_attr_df = compute_attributions('break_start')

Estimating Shapley Values. Average change of Shapley values in run 4 (20 evaluated permutations): -46.46970146169086%: 100%|██████████| 1/1 [00:05<00:00,  5.09s/it]
Estimating Shapley Values. Average change of Shapley values in run 11 (55 evaluated permutations): -10.357966337601544%: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]
Estimating Shapley Values. Average change of Shapley values in run 5 (25 evaluated permutations): -27.84816853420845%: 100%|██████████| 1/1 [00:00<00:00,  1.33it/s]
Estimating Shapley Values. Average change of Shapley values in run 7 (35 evaluated permutations): -25.3988886040801%: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]
Estimating Shapley Values. Average change of Shapley values in run 3 (15 evaluated permutations): -606.3152108585061%: 100%|██████████| 1/1 [00:00<00:00,  2.00it/s]
Estimating Shapley Values. Average change of Shapley values in run 5 (25 evaluated permutations): -19.10091057278802%: 100%|██████████| 1/1 [00:00<00:00,  1.36it/s]
Estimatin

In [43]:
for column in nodes:
    if column == "start":
        continue
    
    test_result = stats.ttest_ind(break_start_attr_df["start"].to_list(),
                                  break_start_attr_df[column].to_list(),
                                  equal_var=False)
    
    print(f"{column}: p-value = {test_result.pvalue}")



count: p-value = 5.393668662864911e-11
gccount1: p-value = 6.172298055128059e-07
gccount2: p-value = 8.388800744130655e-08
atcount1: p-value = 5.332619036655338e-11
atcount2: p-value = 5.436655333903617e-11
gcsum: p-value = 5.4860988240733004e-11
atsum: p-value = 5.3212970356196934e-11
gcratio: p-value = 1.8893114946974004e-31


In [44]:
df = load_dataset('break_start', 1)
df

,gcratio,atsum,gcsum,gccount1,count,start,gccount2,atcount1,atcount2
0,0.401522,708.0,475.0,339.0,52.0,180.0,136.0,550.0,158.0
1,0.397930,640.0,423.0,316.0,56.0,181.0,107.0,513.0,127.0
2,0.397930,640.0,423.0,316.0,92.0,181.0,107.0,513.0,127.0
3,0.397930,640.0,423.0,316.0,84.0,181.0,107.0,513.0,127.0
4,0.376910,367.0,222.0,222.0,68.0,185.0,0.0,367.0,0.0
5,0.376910,367.0,222.0,222.0,88.0,185.0,0.0,367.0,0.0
6,0.382746,508.0,315.0,267.0,88.0,183.0,48.0,442.0,66.0
7,0.382746,508.0,315.0,267.0,90.0,183.0,48.0,442.0,66.0
8,0.382746,508.0,315.0,267.0,51.0,183.0,48.0,442.0,66.0
9,0.393425,572.0,371.0,292.0,81.0,182.0,79.0,477.0,95.0


In [37]:
df = load_dataset('normal', 1)
df

,gcratio,atsum,gcsum,gccount1,count,start,gccount2,atcount1,atcount2
0,0.406410,3704.0,2536.0,1204.0,51.0,105.0,1332.0,1916.0,1788.0
1,0.410170,5765.0,4009.0,1971.0,84.0,107.0,2038.0,3129.0,2636.0
2,0.410952,4948.0,3452.0,1638.0,69.0,98.0,1814.0,2562.0,2386.0
3,0.410311,4175.0,2905.0,1380.0,58.0,94.0,1525.0,2160.0,2015.0
4,0.405556,3852.0,2628.0,1249.0,53.0,105.0,1379.0,1991.0,1861.0
5,0.411071,4947.0,3453.0,1634.0,69.0,93.0,1819.0,2566.0,2381.0
6,0.408021,5683.0,3917.0,1863.0,79.0,93.0,2054.0,2937.0,2746.0
7,0.408874,5462.0,3778.0,1799.0,76.0,100.0,1979.0,2821.0,2641.0
8,0.407564,4621.0,3179.0,1507.0,64.0,105.0,1672.0,2393.0,2228.0
9,0.407671,6316.0,4347.0,2175.0,95.0,101.0,2172.0,3454.0,2862.0


In [21]:
normal_df = load_dataset('normal', 3)
break_start_df = load_dataset('break_start', 3)
attributions = gcm.distribution_change(causal_model, normal_df, break_start_df, 'gcratio')

Estimating Shapley Values. Average change of Shapley values in run 20 (100 evaluated permutations): -9.473660227904112%: 100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [22]:
attributions

{'atcount1': -0.005556603389351271,
 'atcount2': -0.0015950600008409722,
 'atsum': 0.0007885089332568149,
 'count': 0.0053672867997377385,
 'gccount1': 0.9083775155448677,
 'gccount2': 0.8845490696216372,
 'gcratio': -0.004472339636299566,
 'gcsum': 0.007280188158339058,
 'start': 1.6299997378532998}

In [5]:
# concatenator breaks the audit log: https://github.com/scipipe/scipipe/issues/127
# so we need to trace audit up to concatenator
# as well as each audit files leading up to concatenator
# annoying, but alas!
audit_files = ["gcratio.txt.audit.json",
               "chry.fa.gccnt1.audit.json", "chry.fa.gccnt2.audit.json",
               "chry.fa.atcnt1.audit.json", "chry.fa.atcnt2.audit.json"]

raw_data = {}
for final_audit_file in audit_files:
    with open(os.path.join(base_dir, "test", final_audit_file)) as f:
        audit_data = json.load(f)
    
    audit_data_queue = [audit_data]
    while audit_data_queue:
        audit_data = audit_data_queue.pop(0)
        if audit_data["ProcessName"] in nodes:
            # we assume all our nodes output exactly one file
            # Big Assumption!
            data_file_name = next(iter(audit_data["OutFiles"].values()))
            with open(os.path.join(base_dir, "test", data_file_name)) as data_file:
                line = data_file.readline()

            # calc may output tilde for division
            # so we need to remove it, plus any whitespaces
            value = float(line.strip().strip('~'))
            raw_data[audit_data["ProcessName"]] = value
        
        for upstream_data in audit_data["Upstream"].values():
            if upstream_data:
                audit_data_queue.append(upstream_data)


In [6]:
raw_data

{'gcratio': 0.3908614575625491,
 'atsum': 9056162.0,
 'gcsum': 5811001.0,
 'gccount1': 5811001.0,
 'count': 300000.0,
 'start': 1.0,
 'gccount2': 0.0,
 'atcount1': 9056162.0,
 'atcount2': 0.0}

In [7]:
all_records = []

for i in range(1, 21):
    raw_data = {}
    for final_audit_file in audit_files:
        with open(os.path.join(base_dir, "test1", str(i), final_audit_file)) as f:
            audit_data = json.load(f)
        
        audit_data_queue = [audit_data]
        while audit_data_queue:
            audit_data = audit_data_queue.pop(0)
            if audit_data["ProcessName"] in nodes:
                # we assume all our nodes output exactly one file
                # Big Assumption!
                data_file_name = next(iter(audit_data["OutFiles"].values()))
                with open(os.path.join(base_dir, "test1", str(i), data_file_name)) as data_file:
                    line = data_file.readline()

                # calc may output tilde for division
                # so we need to remove it, plus any whitespaces
                value = float(line.strip().strip('~'))
                raw_data[audit_data["ProcessName"]] = value
            
            for upstream_data in audit_data["Upstream"].values():
                if upstream_data:
                    audit_data_queue.append(upstream_data)
    
    all_records.append(raw_data)

df = pd.DataFrame(all_records)


In [8]:
# break start
df

,gcratio,atsum,gcsum,gccount1,count,start,gccount2,atcount1,atcount2
0,0.401522,708.0,475.0,339.0,81.0,180.0,136.0,550.0,158.0
1,0.401522,708.0,475.0,339.0,67.0,180.0,136.0,550.0,158.0
2,0.383795,289.0,180.0,180.0,64.0,187.0,0.0,289.0,0.0
3,0.387543,177.0,112.0,112.0,51.0,190.0,0.0,177.0,0.0
4,0.376956,438.0,265.0,243.0,92.0,184.0,22.0,406.0,32.0
5,0.382746,508.0,315.0,267.0,52.0,183.0,48.0,442.0,66.0
6,0.391198,249.0,160.0,160.0,67.0,188.0,0.0,249.0,0.0
7,0.383795,289.0,180.0,180.0,72.0,187.0,0.0,289.0,0.0
8,0.382746,508.0,315.0,267.0,50.0,183.0,48.0,442.0,66.0
9,0.393425,572.0,371.0,292.0,85.0,182.0,79.0,477.0,95.0


In [9]:
# break count
df

,gcratio,atsum,gcsum,gccount1,count,start,gccount2,atcount1,atcount2
0,0.401522,708.0,475.0,339.0,81.0,180.0,136.0,550.0,158.0
1,0.401522,708.0,475.0,339.0,67.0,180.0,136.0,550.0,158.0
2,0.383795,289.0,180.0,180.0,64.0,187.0,0.0,289.0,0.0
3,0.387543,177.0,112.0,112.0,51.0,190.0,0.0,177.0,0.0
4,0.376956,438.0,265.0,243.0,92.0,184.0,22.0,406.0,32.0
5,0.382746,508.0,315.0,267.0,52.0,183.0,48.0,442.0,66.0
6,0.391198,249.0,160.0,160.0,67.0,188.0,0.0,249.0,0.0
7,0.383795,289.0,180.0,180.0,72.0,187.0,0.0,289.0,0.0
8,0.382746,508.0,315.0,267.0,50.0,183.0,48.0,442.0,66.0
9,0.393425,572.0,371.0,292.0,85.0,182.0,79.0,477.0,95.0


In [10]:
# normal
df

,gcratio,atsum,gcsum,gccount1,count,start,gccount2,atcount1,atcount2
0,0.401522,708.0,475.0,339.0,81.0,180.0,136.0,550.0,158.0
1,0.401522,708.0,475.0,339.0,67.0,180.0,136.0,550.0,158.0
2,0.383795,289.0,180.0,180.0,64.0,187.0,0.0,289.0,0.0
3,0.387543,177.0,112.0,112.0,51.0,190.0,0.0,177.0,0.0
4,0.376956,438.0,265.0,243.0,92.0,184.0,22.0,406.0,32.0
5,0.382746,508.0,315.0,267.0,52.0,183.0,48.0,442.0,66.0
6,0.391198,249.0,160.0,160.0,67.0,188.0,0.0,249.0,0.0
7,0.383795,289.0,180.0,180.0,72.0,187.0,0.0,289.0,0.0
8,0.382746,508.0,315.0,267.0,50.0,183.0,48.0,442.0,66.0
9,0.393425,572.0,371.0,292.0,85.0,182.0,79.0,477.0,95.0
